<p style="text-align: center;font-size: 40pt">Use case 2 - Power Plant Inspection</p>

# Overview 

Requirements
- [Error minimization](4-lesson_error_minimization.ipynb)

Objectives of this lesson:
- give an example of application for point cloud registration

# Introduction
In collaboration with Alstom Inspection Robotics (AIR), prototypes were developed for the inspection and maintenance of industrial plants.
Some inspection tasks need to move inspection tools in environments that are difficult to access by human due to dimensional, temperature or air quality constraints.
The use of mobile systems for inspection can not only deal with those constraints, but also can reduce the time and costs of inspections.
This would, for example, allow for the inspection of critical pieces of equipment on location, without the need to dismantle any structures. Similarly, the installation of scaffolding around a structure becomes unnecessary, thus saving inspection time.
The typical environments encountered during inspection procedures are confined spaces (indoors) with well structured, static surfaces.
<p id="robotMagnebike" style="text-align: center;">
    <img src="images/robot_magnebike.jpg" width="50%"/> <br/>
    <b>Figure 3.11:</b> Three prototypes of chest inspection robots without the sensors, in a mock-up of a steam chest.
	The sensor was not installed at the time of the photograph.
</p>

# Platform
For the specific task of steam chest inspection, a robot was developed with high mobility capabilities and a compact size [[Tâche et al.,
2009]](https://onlinelibrary.wiley.com/doi/abs/10.1002/rob.20296).
The robot, named Magnebike, moved around a metallic, cylindric environment by using its two magnetic wheels positioned in the same configuration as a bike ([Figure 3.11](#robotMagnebike)).
Those specialized wheels coupled with small lever arms allowed the platform to move up-side-down, pass 90$^\circ$ edges and navigate in high curvature tube.
A considerable amount of effort were invested in reducing the size of the platform, which ultimately gave a small volume of 0.006 m$^3$ and a weight of 0.34 kg.
The platform was the slowest presented in this section, with a maximal velocity value of only 0.045 m/s.
The robot gathered high-resolution scans (340,000 points) with a refresh time of 50 s.
The 3D scans were assembled from a 2D Hokuyo URG-04LX.
Pre-alignment of the scans were ensured by wheel odometry, which displays virtually no slip because of the magnetic force hold the wheels on the surfaces.
An IMU was used in conjunction with the odometry to cope with the 3D nature of the motion [[Tâche et al., 2011]](https://onlinelibrary.wiley.com/doi/abs/10.1002/rob.20361).
The main sources of pre-alignment errors were: (1) motions perpendicular to the gravity vector that were not observable with the sensor used, and (2) the rapid cumulation of errors by the low-cost IMU used on the slow-moving robot.

# Solution
During inspection, the robot is intended to be tethered for safety reason, which solved the communication problem between the embedded computer and a faster system used as control station.
Because the system must not damage the inspected structure in any case, it traveled slowly in the environment, reducing the pressure on the real-time requirement for the registration.
However, the operator might not have a visual contact with the robot at all times during inspection.
Therefore, the map resolution must be high enough to detect obstacles and holes during remote operations.
The number of points produced by the scanner was significantly larger than strictly necessary for a proper registration.
To reduce rapidly this number of points, we randomly removed 90 % of the point as soon as the scan were recorded.
We used a maximal density of 20000 points per m$^3$ to cope with the small size of inspected environments.
For the registration, we did not use any pre-alignment to test the worst case scenario (i.e., when the rotation is not observable by the IMU).
This forced us to extend the maximal matching distance to 0.5 m, even if a scan was taken roughly at every 0.30 m.
Given that scans are taken by an operator at a fix and short interval, we used an outlier ratio of 80 %.
The complete list of modules used with their main parameters are listed in [Table 3.4](#icpConfigAppMagnebike).
<p id="icpConfigAppMagnebike" style="text-align: center;margin-left: 15%;margin-right: 15%;">
    <b>Table 3.4:</b> Configuration of the ICP chain for the Magnebike mapping applications.
    The definition of the column <em>Step</em> follows previous lessons. 
    The names used in the column <em>Module</em> refer to specific implementation documented in the open source library <tt>libpointmatcher</tt>.
</p>

|    *Step*   |            *Module*            | *Description* |
|:-----------:|:------------------------------:|:-------------:|
| $$DF_{read}$$ |     <tt>RandomSampling</tt>    | Keep randomly 10 % of the points. |
|             | <tt>SamplingSurfaceNormal</tt> | Keep 80 % of the points, while extracting surface normals based on 20 NN. |
|             |  <tt>ObservationDirection</tt> | Add vector pointing toward the origin of the sensor. |
|             |     <tt>OrientNormals</tt>     | Orient surface normals toward the observation direction. |
|             |       <tt>MaxDensity</tt>      | Subsample to keep point with density of 20000 pts/m$^3$. |
|  $$DF_{ref}$$ |     <tt>SurfaceNormal</tt>     | Compute normal and density with 20 NN and an approximation factor $\epsilon=3.16$. |
|             |       <tt>MaxDensity</tt>      | Subsample to keep point with density of 20000 pts/m$^3$. |
|      MF     |         <tt>KDTree</tt>        | Use an approximate kD-tree with a maximum matching distance of 0.5 m and an approximation factor of $\epsilon=3.16$. |
|      OF     |      <tt>TrimmedDist</tt>      | Keep 80 % closest paired points. |
|             |     <tt>SurfaceNormal</tt>     | Remove paired points with normals angle larger than 50$^\circ$. |
|      EM     |      <tt>PointToPlane</tt>     | Objective function using point-to-plane error. |
|      TC     |      <tt>Differential</tt>     | Stop after a minimum error below 0.01 m and 0.001 rad. |
|             |        <tt>Counter</tt>        | Stop after the iteration count reached 100. |
|             |         <tt>Bound</tt>         | Stop if transformation increases beyond 5.0 m and 0.8 rad. |
<p style="text-align: center;margin-left: 15%;margin-right: 15%;font-size:10px;">
    <em>Legend</em>: $DF_{read}$ = Data Filters for readings, $DF_{ref}$ = Data Filters for references, MF = Matching Function, OF = Outlier Filters, EM = Error Minimizer, TC = Transformation Checker.
</p>

# Results
To test the mapping capability of the platform, a real steam chest was made available by AIR. This part was actually removed from a power plant for reparation purpose.
Multiple inspection runs were executed, each run starting from one of the seven entry points ([Figure 3.12](#resultsSteamChest)).
We only present here the results from the longest path since it covered the entire environment.
The robot started on one side of the steam chest, situated on the left of [Figure 3.12](#resultsSteamChest).
Each 3D scan was taken on a stop-and-go strategy at every 0.1 m.
The total path covered a distance of 5.8 m for a total of 59 scans.
All runs were registered offline approximately 10 times faster than the rate at which they were recorded.
Based on this reconstruction, it was possible to have full 3D path planning and navigation in this environment [[Stumm et al., 2012]](https://journals.sagepub.com/doi/abs/10.1177/0278364912461537).
<p id="resultsSteamChest" style="text-align: center;">
    <img src="images/steamChestMap.png" width="66%"/>
    <img src="images/steamChest.png" width="43.5%"/> <br/>
    <b>Figure 3.12:</b> Deployment results of Magnebike in a real steam chest.
    <em>Top</em>: Cut view of the reconstructed environment. 
    The light gray line correspond to the path of the robot, with the sphere being the positions where the robot stopped to take a 3D scan.
    The colors of the map follow the discreet time (from 0 to 58) at which the scans were taken.
    <em>Bottom</em>: The actual steam chest removed for maintenance.
</p>

# Conclusion
You should do the following activities to enhance your understanding of the concepts viewed in this lesson:
- modify the markdown by adding your own notes using `> my notes`; and
- complete the tables [Symbol definitions](#Symbol-definitions) and [Glossary](#Glossary) and add your own definitions.

Parallel lessons:
- [Use case 1](5-lesson_use_case_1.ipynb)
- [Use case 3](5-lesson_use_case_3.ipynb)

Next lesson:
- [Registration challenges](6-lesson_challenges.ipynb)

## Symbol definitions

| Symbol             | Definition  |
|--------------------|-------------|
|                    |             |

## Glossary

| English   | Français   | Definition |
|-----------|------------|------------|
|           |            |            |